<a href="https://colab.research.google.com/github/nman7/coursera/blob/master/courseraUae.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**->IBM DATASCIENCE CAPSTONE PROJECT**

  **-> WEEK5**

**->FIND THE BEST AREA TO OPEN A NEW GAS STATION IN DUBAI CITY.**



1.   **obtain data of dubai suburbs from wikipedia page**

2.   **get geographical coordinates of each suburb**

3.   **get top venues around each suburb using foursquareapi**

4.   **analyze and  cluster suburbs**

5.   **find the best cluster**













---


**import dependecy**



In [488]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files
!pip install geocoder
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


**scrapped data from wikipedia**

In [0]:
data = pd.read_csv('saudiarabia.csv')

In [0]:
data.tail()
data = data[['neighbourhood']]

In [491]:
data.head()
print(data.shape)

(116, 1)


**getting latitude and longitude **

In [0]:

def get_latlng(x):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Dubai,uae'.format(x))
        lat_lng_coords = g.latlng
    return lat_lng_coords


In [0]:

coords = [ get_latlng(x) for x in data["neighbourhood"].tolist() ]


In [494]:
print(coords)


[[25.17793000000006, 55.23422000000005], [25.26836000000003, 55.32634000000007], [25.278350000000046, 55.308170000000075], [25.27177000000006, 55.33762000000007], [25.164780000000064, 55.270800000000065], [25.126490000000047, 55.19817000000006], [25.057650000000024, 55.27140000000003], [25.238090000000057, 55.29779000000008], [25.187970000000064, 55.26273000000003], [25.245160000000055, 55.29333000000008], [25.122700000000066, 55.22761000000003], [25.220540000000028, 55.34166000000005], [25.08958000000007, 55.23424000000006], [25.272550000000024, 55.30313000000007], [25.15411000000006, 55.340230000000076], [25.21861000000007, 55.26406000000003], [25.195110000000057, 55.40933000000007], [25.085750000000075, 55.163470000000075], [25.300940000000026, 55.34003000000007], [25.27326000000005, 55.44738000000007], [25.145790000000034, 55.209580000000074], [25.182240000000036, 55.345920000000035], [25.179560000000038, 55.23369000000008], [25.24033000000003, 55.32371000000006], [25.2722800000000

In [0]:
cordinates = pd.DataFrame(coords,columns=['Latitudes','Longitudes'])

In [496]:
cordinates.head()

,Latitudes,Longitudes
0,25.17793,55.23422
1,25.26836,55.32634
2,25.27835,55.30817
3,25.27177,55.33762
4,25.16478,55.27080


In [0]:
data['Latitudes'] = cordinates['Latitudes']
data['Longitudes'] = cordinates['Longitudes']

**neighbour hood data with both latitude and longitude**

In [498]:
print(data.head())
data.shape

    neighbourhood  Latitudes  Longitudes
0   Al Safa First   25.17793    55.23422
1   Al Muraqqabat   25.26836    55.32634
2        Al Murar   25.27835    55.30817
3      Al Khabisi   25.27177    55.33762
4  Al Quoz Second   25.16478    55.27080


(116, 3)

**conversion to coordinates using geocode**

In [499]:
# get the coordinates of Kuala Lumpur
address = 'dubai,uae'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Dubai,Uae {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Dubai,Uae 25.0657, 55.1713.


**creating a map and marking the location on map**

In [0]:

# create map of DUBAI using latitude and longitude values
map_kl = folium.Map(location=[latitude, longitude], zoom_start=11)


# add markers to map
for lat, lng, neighborhood in zip(data['Latitudes'], data['Longitudes'], data['neighbourhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_kl)  

    

In [501]:
map_kl

**saving the map**

In [0]:
map_kl.save('map_kl.html')

**foursquare api credentials**

In [503]:

# define Foursquare Credentials and Version
CLIENT_ID = 'L4LBWYV1GC2O0UARZUTD5LZTPRXTMSXLAOMKJVEBXWYICCKO' # your Foursquare ID
CLIENT_SECRET = 'GHFSP4XMGIYNTCY2VGG2NUMPPLTJ4TTMCVWIWBOSYX05M2JJ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: L4LBWYV1GC2O0UARZUTD5LZTPRXTMSXLAOMKJVEBXWYICCKO
CLIENT_SECRET:GHFSP4XMGIYNTCY2VGG2NUMPPLTJ4TTMCVWIWBOSYX05M2JJ


**analyze each location by getting nearby venues info by setting radius around that each location**

In [0]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(data['Latitudes'], data['Longitudes'], data['neighbourhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [505]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(8271, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Al Safa First,25.17793,55.23422,Starbucks (ستاربكس),25.183929,55.233493,Coffee Shop
1,Al Safa First,25.17793,55.23422,Baskin Robbins,25.177470,55.233496,Ice Cream Shop
2,Al Safa First,25.17793,55.23422,Yamanote Atelier,25.185414,55.238833,Dessert Shop
3,Al Safa First,25.17793,55.23422,PizzaExpress,25.179777,55.235074,Italian Restaurant
4,Al Safa First,25.17793,55.23422,dip 'n dip,25.186094,55.229010,Dessert Shop


** number of venues of each category**

In [506]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Abu Hail,100,100,100,100,100,100
Al Bada,100,100,100,100,100,100
Al Baraha,100,100,100,100,100,100
Al Barsha First,100,100,100,100,100,100
Al Barsha Second,100,100,100,100,100,100
Al Barsha South First,20,20,20,20,20,20
Al Barsha South Second,19,19,19,19,19,19
Al Barsha South Third,45,45,45,45,45,45
Al Barsha Third,100,100,100,100,100,100


In [507]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 303 uniques categories.


In [508]:
venues_df['VenueCategory'].unique()[:2000]

array(['Coffee Shop', 'Ice Cream Shop', 'Dessert Shop',
       'Italian Restaurant', 'Café', 'Salon / Barbershop',
       'Indoor Play Area', 'Juice Bar', 'Beach', 'Bakery',
       'Egyptian Restaurant', 'Seafood Restaurant', 'Health Food Store',
       'Massage Studio', 'Supermarket', 'Afghan Restaurant', 'Resort',
       'Arcade', 'Sporting Goods Shop', 'Mediterranean Restaurant',
       'Burger Joint', 'Middle Eastern Restaurant', 'Motorcycle Shop',
       'Park', 'Peruvian Restaurant', 'Restaurant', 'Donut Shop', 'Plaza',
       'Gym / Fitness Center', 'Boutique', 'Grocery Store',
       'Breakfast Spot', 'Furniture / Home Store', 'Indian Restaurant',
       'Turkish Restaurant', 'Korean Restaurant', 'Wings Joint',
       'Shawarma Place', 'Department Store', 'Japanese Restaurant',
       'Mexican Restaurant', 'Fast Food Restaurant', 'Spa', 'Canal',
       'Moroccan Restaurant', 'American Restaurant', 'Kids Store',
       'Hookah Bar', 'Clothing Store', 'Fish Market', "Women's Stor

checking if gas station exist

In [546]:
"Gas Station" in venues_df['VenueCategory'].unique()

True

In [547]:
# one hot encoding
data_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
data_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [data_onehot.columns[-1]] + list(data_onehot.columns[:-1])
data_onehot = data_onehot[fixed_columns]

print(data_onehot.shape)
data_onehot.head()

(8271, 304)


,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Bavarian Restaurant,Beach,Beach Bar,Bed & Breakfast,Beer Garden,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bistro,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Business Service,Cafeteria,Café,Camera Store,Campground,Canal,Candy Store,Carpet Store,Castle,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Currency Exchange,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dive Shop,Dog Run,Donut Shop,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health & Beauty Service,Health Food Store,Heliport,Historic Site,History Museum,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Housing Development,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Iraqi Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Mobility Store,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Organic Grocery,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Photography Lab,Photography Studio,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Pub,Racetrack,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Seafood Restaurant,Shawarma Place,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Ski Area,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,South Indian Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stables,Stadium,Steakhouse,Supermarket,Surf Spot,Sushi Restaurant,Syrian Restaurant,Taco Place,Tailor Shop,Tapas Restaurant,Tea Room,Tennis Court,Tennis St

In [548]:
data_grouped = data_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(data_grouped.shape)
data_grouped.head()

(116, 304)


,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Bavarian Restaurant,Beach,Beach Bar,Bed & Breakfast,Beer Garden,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bistro,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Business Service,Cafeteria,Café,Camera Store,Campground,Canal,Candy Store,Carpet Store,Castle,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Currency Exchange,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dive Shop,Dog Run,Donut Shop,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health & Beauty Service,Health Food Store,Heliport,Historic Site,History Museum,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Housing Development,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Iraqi Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Mobility Store,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Organic Grocery,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Photography Lab,Photography Studio,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Pub,Racetrack,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Seafood Restaurant,Shawarma Place,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Ski Area,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,South Indian Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stables,Stadium,Steakhouse,Supermarket,Surf Spot,Sushi Restaurant,Syrian Restaurant,Taco Place,Tailor Shop,Tapas Restaurant,Tea Room,Tennis Court,Tennis St

In [549]:
len(kl_grouped[kl_grouped["Gas Station"] > 0])

10

In [0]:
data_gym = data_grouped[["Neighborhoods","Gas Station"]]

In [551]:
data_gym.head()

,Neighborhoods,Gas Station
0,Abu Hail,0.0
1,Al Bada,0.0
2,Al Baraha,0.0
3,Al Barsha First,0.0
4,Al Barsha Second,0.0


In [0]:
# set number of clusters
kclusters = 3
data_clustering = data_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(data_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
data_merged = data_mall.copy()

# add clustering labels
data_merged["Cluster Labels"] = kmeans.labels_

In [0]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
data_merged = data_mall.copy()

# add clustering labels
data_merged["Cluster Labels"] = kmeans.labels_

In [554]:
data_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
data_merged.head()

,Neighborhood,Gym,Cluster Labels
0,Abu Hail,0.03,1
1,Al Bada,0.00,0
2,Al Baraha,0.03,1
3,Al Barsha First,0.00,0
4,Al Barsha Second,0.00,0


In [555]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
data_merged = data_merged.join(data.set_index("neighbourhood"), on="Neighborhood")

print(data_merged.shape)
data_merged.head() # check the last columns!

(116, 5)


,Neighborhood,Gym,Cluster Labels,Latitudes,Longitudes
0,Abu Hail,0.03,1,25.28308,55.33435
1,Al Bada,0.00,0,25.21861,55.26406
2,Al Baraha,0.03,1,25.28280,55.31678
3,Al Barsha First,0.00,0,25.11483,55.19136
4,Al Barsha Second,0.00,0,25.10723,55.20485


In [556]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(data_merged['Latitudes'], data_merged['Longitudes'], data_merged['Neighborhood'], data_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [0]:
#save the map as HTML file
map_clusters.save('map_clusters.html')

In [558]:
kl_merged.loc[kl_merged['Cluster Labels'] == 0]

,Neighborhood,Gym,Cluster Labels
1,Al Bada,0.0,0
3,Al Barsha First,0.0,0
4,Al Barsha Second,0.0,0
8,Al Barsha Third,0.0,0
14,Al Hudaiba,0.0,0
16,Al Jafiliya,0.0,0
17,Al Karama,0.0,0
19,Al Kifaf,0.0,0
22,Al Mankhool,0.0,0
33,Al Quoz First,0.0,0


In [559]:
kl_merged.loc[kl_merged['Cluster Labels'] == 1]

,Neighborhood,Gym,Cluster Labels
6,Al Barsha South Second,0.040000,1
7,Al Barsha South Third,0.040000,1
24,Al Mizhar First,0.050000,1
47,Al Qusais Third,0.042553,1
73,Emirates Hill First,0.040000,1
84,Mirdif,0.040000,1
86,Muhaisanah Second,0.054348,1
88,Muhaisnah First,0.040000,1
93,Nadd Al Shiba Third,0.046512,1
95,Oud Al Muteena Third,0.060000,1


In [560]:
kl_merged.loc[kl_merged['Cluster Labels'] == 2]

,Neighborhood,Gym,Cluster Labels
9,Al Buteen,0.010000,2
10,Al Dhagaya,0.010000,2
13,"Al Hamriya, Dubai",0.010000,2
21,Al Manara,0.010000,2
26,Al Muraqqabat,0.010000,2
27,Al Murar,0.010000,2
32,"Al Nasr, Dubai",0.010000,2
38,Al Quoz Second,0.010000,2
39,Al Quoz Third,0.010000,2
42,Al Qusais Industrial First,0.010000,2


-From the above analysis it can be seen that most of the Gas station are owned in the location marked with purple which highlights that most of them are industrial area of Dubai city,UAE i.e high concentration of Gas-station in cluster 1 with Purple mark and Moderate amount of conentration can be seen in cluster 2 highlighted with sea-green mark and with very less concentration in cluster 0 with the red mark.Aim of this project is  recommended for oil company best location to provide gas-station in dubai.It can be seen that less number of Gasstation are found near the tourist destination, so it is recommended to select area with almost no competiton i.e cluster0 or they can start of with mild competition in cluster2. and it is advised to avoid cluster1 as the intense competition with number of Gas-station in their locality.


